In [1]:
import json
from collections import Counter
import matplotlib.pyplot as plt
import matplotlib
from tqdm import tqdm
import operator
import seaborn as sns
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
import re,string
from nltk.tokenize import sent_tokenize
import spacy
import networkx as nx
from nltk.corpus import stopwords
from spacy import displacy
import pandas as pd
import nltk
from grammaregex import print_tree, match_tree, find_tokens
#https://stackoverflow.com/questions/39763091/how-to-extract-subjects-in-a-sentence-and-their-respective-dependent-phrases
#https://medium.com/@krzysiek89dev/grammaregex-library-regex-like-for-text-mining-49e5706c9c6d#.zgx7odhsc

spacy_d = {"it":"it_core_news_sm",
           "es":"es_core_news_sm",
           "en":"en_core_web_sm",
           "nl":"nl_core_news_sm"}

# SUBJECTS = ["nsubj", "nsubjpass", "csubj", "csubjpass", "agent", "expl"]
# OBJECTS = ["dobj", "dative", "attr", "oprd"]
# ADJECTIVES = ["acomp", "advcl", "advmod", "amod", "appos", "nn", "nmod", "ccomp", "complm",
#               "hmod", "infmod", "xcomp", "rcmod", "poss"," possessive"]
# COMPOUNDS = ["compound"]
# PREPOSITIONS = ["prep"]

/home/ruben/anaconda3/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
base_path = "/media/ruben/Data Drive/react-data/protest/{}".format("carlo-batch-selection")

In [2]:
data = pd.read_csv(open('/media/ruben/FEF44259F44213F5/Users/Ruben/Documents/GitHub/ReACT_GCV/data/images_tables_article_carlo/data-full.csv'),engine='c')
#model = KeyedVectors.load_word2vec_format('/media/ruben/Data Drive/react-data/model.bin',binary=True)

In [38]:
violence_vocab = ['violenza','violenze']
protesters_vocab = ['manifestanti','manifestante','dimostranti','dimostrante','contestatori','contestatore','attivisti','attiviste','facinorosi','facinorosi','militanti']
protests_vocab = "dimostrazione manifestazione manifestazioni dimostrazioni"
forze_vocab
police_vocab = ["forze dell'ordine",'carabinieri','carabineri','carabinierii','carabineiri','polstrada','polizia','poliziotti','carabiniere']

In [3]:
data['sentences'] = [" ".join(str(x).split('||')).replace('||',' ') for x in list(data['sentences'])]

In [40]:
def Parse(keyword_vocab,language):
    nlp = spacy.load(spacy_d[language])
    
    ss=[]

    for c,i in enumerate(data['sentences']):
        if any(w in i for w in keyword_vocab) and data['language'][c] == language:
            sentences = sent_tokenize(i)
            sentences = [s for s in sentences if any(n in s for n in keyword_vocab)]

            for sentence in sentences:
                doc = nlp(sentence)
                docs = [[str(x.text),str(x.dep_),str(x.pos_),[str(child) for child in x.children]] for x in doc]
                ss.append(docs)
    dss = []
    for c,doc in enumerate(ss):
        tmp = [w for w in doc if w[0] in keyword_vocab]
        if len(tmp) > 0:
            for w in tmp:
                dss.append([c,w[0],w[1],w[2]," ".join(w[3])," ".join([x[0] for x in doc]),doc])
    dss = pd.DataFrame(dss,columns="doc word dep pos subj sentence fulldeps".split(' '))
    return dss

def NounAdjCombs(keyword_vocab,language):
    nlp = spacy.load(spacy_d[language])
    ss=[]

    for c,i in enumerate(data['sentences']):
        if any(w in str(i) for w in keyword_vocab) and data['language'][c] == language:
            sentences = sent_tokenize(i)
            sentences = [s for s in sentences if any(n in s for n in keyword_vocab)]

            for sentence in sentences:
                doc = nlp(sentence)
                for i in doc:
                    if i.pos_ in ["NOUN", "PROPN"] and str(i.text) in keyword_vocab:
                        comps = [j for j in i.children if j.pos_ in ["ADJ"]]
                        if comps:
                            for co in comps:
                                ss.append([str(co),str(co.lemma) ,str(i),data['identifier'][c]])
    return pd.DataFrame(ss,columns=['adj','adj_lemma','noun','id'])

In [8]:
nlp = spacy.load(spacy_d['it'])
doc = nlp("un'azione estremamente violenta non è stata accolta con favore dalle autorità")
for i in doc:
    print(i.text,i.pos_)
    [print(j.text,j.pos_,j.dep_) for j in i.children]
    print('----')

un' DET
----
azione NOUN
un' DET det
violenta ADJ amod
----
estremamente ADV
----
violenta ADJ
estremamente ADV advmod
----
non ADV
----
è AUX
----
stata AUX
----
accolta VERB
azione NOUN nsubj:pass
non ADV advmod
è AUX aux
stata AUX aux:pass
favore NOUN obl
autorità NOUN nsubj:pass
----
con ADP
----
favore NOUN
con ADP case
dalle ADJ compound
----
dalle ADJ
----
autorità NOUN
----


In [21]:
s = []

for i in data['sentences']:
    if "forz" in i:
        indi = i.index('forz')
        s.append(i[indi:indi+22])

In [22]:
s

['forze dell’ordine e ma',
 'forze dell’ordine maur',
 'forza delle armi. e’ u',
 'forzar un cambio polít',
 'forza quel particolare',
 'forze dell’ordine maur',
 'forzi , che sono a men',
 'forze di polizia, orga',
 "forze del dell'ordine ",
 'forza del tutto nuova ',
 'forzosa en convoyes es',
 'forze dell’ordine veng',
 'forza di un apparato d',
 'forzi , che sono a men',
 'forza per favorire gli',
 'forza e legato al tavo',
 'forze dell’ordine, car',
 'forze dell’ordine, car',
 'forze dell’ordine in q',
 'forze dell’ordine, car',
 'forzar un cambio polít',
 'forze dell’ordine”. tu',
 'forzi che in tanti abb',
 'forze dell’ordine che ',
 'forze dell’ordine, car',
 'forze dellordine da qu',
 'forzo con il nostro ce',
 'forze della sinistra i',
 'forze dell’ordine si t',
 'forze dell’ordine si t',
 "forze dell'ordine, un ",
 'forzi , che sono a men',
 'forze dellordine da qu',
 "forze del dell'ordine ",
 "forze dell'ordine, un ",
 'forzi , che sono a men',
 'forze dell’ordine ( e ',
 